# Analysis of Imaging Data

## Signal Extraction & Source-Separation

### Import Modules

In [1]:
# FISSA: Signal Extraction & Source-Separation
from AnalysisModules.FissaAnalysis import FissaModule
# Assorted Static Processing Functions
from AnalysisModules.StaticProcessing import smoothTraces_TrialOrg, calculate_dFoF, mergeTraces, detrendTraces

### Instantiation

In [2]:
# Instantiate Fissa Module & Sub-Modules (preparation, experiment (separation), and ProcessedTraces)
Data = FissaModule(data_folder="H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz", index_path = "H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz\\NeuronalIndex.csv")
# Initialize
Data.initializeFissa()

Initialized Fissa


### Extract Traces

In [3]:
Data.extractTraces()

Extracting traces:   0%|          | 0/4 [00:00<?, ?it/s]

Finished extracting raw signals from 1285 ROIs across 4 trials in 4 min, 21 sec.
Passing traces between modules.
Finished module-passing.
Ready for post-processing or source-separation.


### Save Progress

In [4]:
Data.saveFissaPrep()

Saving extracted traces to H:\DEM_Excitatory_Study\DEM2\Retrieval\Imaging\10Hz\prepared.npz
Finished Saving Prep


### Pre-Process Traces

In [5]:
Data.ProcessedTraces.smoothed_raw = smoothTraces_TrialOrg(Data.preparation.raw, niter=50, kappa=150, gamma=0.15)[0]

Smoothing:   0%|          | 0/1285 [00:00<?, ?it/s]

Organizing By Trial:   0%|          | 0/1285 [00:00<?, ?it/s]

### Save Progress

In [6]:
Data.saveProcessedTraces()

### Use Pre-Processed Traces for Source-Separation

In [7]:
Data.preparation.raw = Data.ProcessedTraces.smoothed_raw.copy()
Data.passPrepToFissa()

### Source-Separate Traces

In [8]:
Data.separateTraces()

Passing prepared traces to fissa.
Initiating fissa source-separation


Separating data:   0%|          | 0/1285 [00:00<?, ?it/s]

Finished separating signals from 1285 ROIs across 4 trials in 51 min, 9 sec
Passing traces between modules.
Finished module-passing.
Ready for further analysis.


### Save Progress

In [9]:
Data.saveFissaSep()

Saving results to H:\DEM_Excitatory_Study\DEM2\Retrieval\Imaging\10Hz\separated.npz


### Post-Process Traces

In [10]:
# Calculate Fo/F
Data.ProcessedTraces.dFoF_result = calculate_dFoF(Data.experiment.result, Data.frame_rate, raw=Data.preparation.raw, merge_after=False)
# Condense the ROI Traces for each Trial into a Single Matrix
Data.ProcessedTraces.merged_dFoF_result = mergeTraces(Data.ProcessedTraces.dFoF_result)
# Detrend the Traces
Data.ProcessedTraces.detrended_merged_dFoF_result = detrendTraces(Data.ProcessedTraces.merged_dFoF_result, order=4, plot=False)

Calculating Δf/f0  (using single f0 across all trials using f0 derived from raw traces during calculations)


Calculating Δf/f0:   0%|          | 0/1285 [00:00<?, ?it/s]

Merging Traces Across Trials:   0%|          | 0/1285 [00:00<?, ?it/s]

Detrending:   0%|          | 0/1285 [00:00<?, ?it/s]

### Save Progress

In [11]:
Data.saveProcessedTraces()

## Infer Firing Rates & Approximate Spikes

### Import Modules

In [12]:
from AnalysisModules.CascadeAnalysis import CascadeModule
from AnalysisModules.StaticProcessing import calculateFiringRate
from AnalysisModules.StaticUtilities import pullModels

	YAML reader installed (version 0.17.17).


Using TensorFlow backend.


	Keras installed (version 2.3.1).
	Tensorflow installed (version 2.3.0).


### Instantiation

In [13]:
# Pass the Post-Processed Traces
Data.Cascade = CascadeModule(Data.ProcessedTraces.detrended_merged_dFoF_result, Data.frame_rate, model_folder="C:\\ProgramData\\Anaconda3\\envs\\suite2p\\Pretrained_models")

### Model Selection

In [14]:
# Pull Available Models
list_of_models = pullModels(Data.Cascade.model_folder)
# Select Model
Data.Cascade.model_name = list_of_models[19]

### Infer Firing Rates

In [15]:
# Infer Spike Probability
Data.Cascade.predictSpikeProb()
# Calculate Firing Rates
Data.Cascade.ProcessedInferences.firing_rates = calculateFiringRate(Data.Cascade.spike_prob, Data.Cascade.frame_rate)

The configuration file "config.yaml" can not be found at the location "C:\ProgramData\Anaconda3\envs\suite2p\AnalysisModules\Pretrained_models\Global_EXC_10Hz_smoothing100ms\config.yaml".
You have provided the model "Global_EXC_10Hz_smoothing100ms" at the absolute or relative path "Pretrained_models".
Please check if there is a folder for model "Global_EXC_10Hz_smoothing100ms" at the location "C:\ProgramData\Anaconda3\envs\suite2p\AnalysisModules\Pretrained_models".


Exception: The configuration file "config.yaml" can not be found at the location "C:\ProgramData\Anaconda3\envs\suite2p\AnalysisModules\Pretrained_models\Global_EXC_10Hz_smoothing100ms\config.yaml".
You have provided the model "Global_EXC_10Hz_smoothing100ms" at the absolute or relative path "Pretrained_models".
Please check if there is a folder for model "Global_EXC_10Hz_smoothing100ms" at the location "C:\ProgramData\Anaconda3\envs\suite2p\AnalysisModules\Pretrained_models".

### Save Progress

In [ ]:
Data.Cascade.exportSpikeProb(Data.output_folder)

### Infer Discrete Spike Times

In [ ]:
Data.Cascade.inferDiscreteSpikes()

### Save Progress

In [ ]:
Data.Cascade.exportSpikeInference(Data.output_folder)